In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from scipy.spatial import Voronoi
import vispy as vp


import tyssue
from tyssue.core.sheet import Sheet
from tyssue.core.objects import Epithelium
from tyssue.geometry import bulk_geometry as bgeom


In [2]:
def hexa_grid3d(nx, ny, nz, distx=1., disty=1., distz=1., noise=None):
    """Creates an hexagonal grid of points
    """
    cz, cy, cx = np.mgrid[0:nz, 0:ny, 0:nx]
    cx = cx.astype(np.float)
    cy = cy.astype(np.float)
    cz = cz.astype(np.float)
    cx[1::2, :] += 0.5
    cy[::2, :] += 0.5
    
    
    centers = np.vstack([cx.flatten(),
                         cy.flatten(), cz.flatten()]).T
    centers[:, 0] *= distx
    centers[:, 1] *= disty
    centers[:, 2] *= distz
    if noise is not None:
        pos_noise = np.random.normal(scale=noise, size=centers.shape)
        centers += pos_noise
    return centers


In [3]:
grid = hexa_grid3d(3, 3, 3)

In [4]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(*grid.T, c='k', marker='o')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

/home/guillaume/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [5]:
voro = Voronoi(grid)

In [6]:
voro.points.shape

(27, 3)

In [7]:
el_idx = []

for f_idx, (rv, rp) in enumerate(
    zip(voro.ridge_vertices,
        voro.ridge_points)):
    
    if -1 in rv:
        continue
    face_verts = voro.vertices[rv]
    f_center = face_verts.mean(axis=0)
    c0 = voro.points[rp[0]]
    ctof = f_center - c0

    for rv0, rv1 in zip(rv, np.roll(rv, 1, axis=0)):
        fv0 = voro.vertices[rv0]
        fv1 = voro.vertices[rv1]
        edge_v = fv1 - fv0
        fto0 = fv0 - f_center
        normal = np.cross(fto0, edge_v)
        dotp = ctof @ normal
        if np.sign(dotp) > 0:
            el_idx.append([rv0, rv1, f_idx, rp[0]])
            el_idx.append([rv1, rv0, f_idx, rp[1]])
        else:
            el_idx.append([rv1, rv0, f_idx, rp[0]])
            el_idx.append([rv0, rv1, f_idx, rp[1]])

el_idx = np.array(el_idx)

In [8]:
from tyssue.core.generation import data_dicts, make_df

In [9]:
cell_data = {
    'volume': (0., np.float),
    'num_faces': (6, int),
    'area': (0.0, float),
    'x': (0.0, float),
    'y': (0.0, float),
    'z': (0.0, float)
    }

data_dicts['cell'] = cell_data
data_dicts['je']['cell'] = (0, np.int)

In [10]:
coords = ['x', 'y', 'z']

je_idx = pd.Index(range(el_idx.shape[0]), name='je')
je_df = make_df(je_idx, data_dicts['je'])
for i, elem in enumerate(['srce', 'trgt', 'face', 'cell']):
    je_df[elem] = el_idx[:, i]


jv_idx = pd.Index(range(voro.vertices.shape[0]), name='jv')
jv_df = make_df(jv_idx, data_dicts['jv'])
jv_df[coords] = voro.vertices

cell_idx = pd.Index(range(voro.points.shape[0]), name='cell')
cell_df = make_df(cell_idx, data_dicts['cell'])
cell_df[coords] = voro.points

face_idx = pd.Index(range(f_idx), name='face')
face_df = make_df(face_idx, data_dicts['cell'])
je_df.sort_values(by='cell', inplace=True)

In [11]:
datasets= {
    'jv': jv_df,
    'je': je_df,
    'face': face_df,
    'cell': cell_df,
}
eptm = Epithelium('start3D', datasets)

In [12]:
from vispy import app, gloo, visuals, scene
from vispy.geometry import MeshData

In [13]:
eptm.je_mindex.names

FrozenList(['srce', 'trgt', 'face', 'cell'])

In [18]:
coords = ['x', 'y', 'z']
upcast_pos = eptm.upcast_face(eptm.jv_df[coords])
upcast_pos.index = eptm.je_mindex
face_df[coords] = upcast_pos.groupby(level='face').mean()

In [ ]:
canvas = scene.SceneCanvas(keys='interactive', show=True)

grid = canvas.central_widget.add_grid()
view = grid.add_view(0, 1)
#view = canvas.central_widget.add_view()
view.camera =  'turntable'
view.camera.aspect = 1

cell_color = vp.color.ColorArray(cell_style['color'], alpha=0.5)
cell_color = cell_color.rgba.repeat(eptm.Nf, axis=0)

edge_color = vp.color.ColorArray(edge_arrow_style['color'], alpha=1)

view.bgcolor = vp.color.Color('#ffffff')


mesh = vp.scene.visuals.Mesh(vertices=vertices,
                             faces=faces,
                             face_colors=cell_color)

wire_pos = vertices[eptm.Nc:].copy()
wire_pos[:, 2] += 0.1

wire = vp.scene.visuals.Line(pos=wire_pos, 
                             connect=faces[:, :2]-eptm.Nc,
                             color=edge_color,
                             width=20)

view.add(mesh)
view.add(wire)
view.camera.set_range()
canvas.show()

#vp_widget = VispyWidget()
#vp_widget.set_canvas(canvas)



app.run()